In [44]:
!pip install scikit-learn
!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import seaborn as sns

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\jiaro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\jiaro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [89]:
pierce = pd.read_csv(r"C:\Users\jiaro\.vscode\happiness-regression\pierce_county_house_sales.csv")


In [64]:
pierce.head()

,sale_date,sale_price,house_square_feet,attic_finished_square_feet,basement_square_feet,attached_garage_square_feet,detached_garage_square_feet,fireplaces,hvac_description,exterior,interior,stories,roof_cover,year_built,bedrooms,bathrooms,waterfront_type,view_quality,utility_sewer
0,2020-01-02,380000,2157,0,0,528,0,0,Warm and Cool Air Zone,Frame Siding,Drywall,2,Composition Shingle,1993,3,3,NaN,NaN,SEWER/SEPTIC INSTALLED
1,2020-01-02,383000,2238,0,0,396,0,0,Forced Air,Frame Siding,Drywall,2,Composition Shingle,2017,4,3,NaN,NaN,SEWER/SEPTIC INSTALLED
2,2020-01-01,419900,1452,0,0,0,0,1,Forced Air,Frame Siding,Drywall,1,Built Up Rock,1965,3,2,NaN,NaN,SEWER/SEPTIC INSTALLED
3,2020-01-02,251660,832,0,0,0,0,0,Forced Air,Frame Vinyl,Drywall,1,Composition Shingle,1919,2,1,NaN,NaN,SEWER/SEPTIC INSTALLED
4,2020-01-02,500000,1780,0,384,484,0,1,Forced Air,Frame Siding,Drywall,1,Composition Shingle,1955,3,2,NaN,NaN,SEWER/SEPTIC INSTALLED


In [90]:

pierce['waterfront_type'].fillna('No Waterfront', inplace=True)
pierce['view_quality'].fillna('No View', inplace=True)


pierce['Log_Price'] = np.log(pierce['sale_price'])

pierce['sale_date'] = pd.to_datetime(pierce['sale_date'])

pierce['sale_date_numeric'] = (pd.Timestamp('today') - pierce['sale_date']) // pd.Timedelta(days=1)

pierce = pierce.drop(columns = ['sale_date','stories', 'interior', 'exterior', 'roof_cover'])
pierce = pierce.dropna()

pierce['garage_square_feet'] = pierce['attached_garage_square_feet'] + pierce['detached_garage_square_feet']

numeric_features = ['sale_date_numeric', 'garage_square_feet',
                    'house_square_feet', 'basement_square_feet', 'fireplaces', 'year_built',
                    'bedrooms', 'bathrooms']
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(pierce[numeric_features])
scaled_numeric = pd.DataFrame(scaled_array, columns=numeric_features, index=pierce.index)

pierce_categorical = pd.get_dummies(pierce[['waterfront_type', 'view_quality', 'hvac_description', 'utility_sewer']], drop_first=True)



C:\Users\jiaro\AppData\Local\Temp\ipykernel_8088\3985096008.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pierce['waterfront_type'].fillna('No Waterfront', inplace=True)
C:\Users\jiaro\AppData\Local\Temp\ipykernel_8088\3985096008.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [92]:
X_combined = pd.concat([scaled_numeric, pierce_categorical], axis=1)
y = pierce['Log_Price']

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

rmse_log = np.sqrt(mean_squared_error(y_test, y_pred))
r2_log = r2_score(y_test, y_pred)
print(f"Test RMSE (log dollars): {rmse_log}")
print(f"Test R^2 (log dollars): {r2_log}")

# Back-transform predictions to actual dollars
#y_test_actual = np.exp(y_test)
#y_pred_actual = np.exp(y_pred)

#rmse_actual = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
#print(f"Test RMSE (actual dollars): ${rmse_actual:,.2f}")


train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = train_mse**0.5
train_r2 = r2_score(y_train, y_train_pred)

print(f"train rmse: {train_rmse}")
print(f"train r2: {train_r2}")

Test RMSE (log dollars): 0.28525595738046045
Test R^2 (log dollars): 0.517882021596981
train rmse: 0.30906796209435283
train r2: 0.4824327516256478
